In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import torch
#from torch import nn

from numba import njit
from tqdm import tqdm

from joblib import Parallel, delayed
from functools import lru_cache
#from multiprocessing import Pool

In [2]:
sample = pd.read_csv('original_data/sample_submission.csv')
sample

,image_id,InChI
0,00000d2a601c,InChI=1S/H2O/h1H2
1,00001f7fc849,InChI=1S/H2O/h1H2
2,000037687605,InChI=1S/H2O/h1H2
3,00004b6d55b6,InChI=1S/H2O/h1H2
4,00004df0fe53,InChI=1S/H2O/h1H2
...,...,...
1616102,ffffcdb2e39e,InChI=1S/H2O/h1H2
1616103,ffffcfddd770,InChI=1S/H2O/h1H2
1616104,ffffe4ab06b2,InChI=1S/H2O/h1H2
1616105,ffffec4033ec,InChI=1S/H2O/h1H2


In [8]:
!ls -lR original_data/test/ | grep '.png' | wc -l

1616107


### Total: 1616107 test pictures.

#### Submission 1.

Submitting their file with constant weights results in score 110. 

## Randomly chosing from train

In [9]:
train = pd.read_csv('original_data/train_labels.csv')
train

,image_id,InChI
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...
3,000026b49b7e,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...
4,000026fc6c36,InChI=1S/C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7...
...,...,...
2424181,ffffe824f539,InChI=1S/C10H12F2N2O3/c1-17-7-2-5(3-8(15)16)9(...
2424182,ffffea1ebdfa,InChI=1S/C19H20F2N4O3S/c1-10(8-20)28-14-7-12(2...
2424183,ffffee2b4534,InChI=1S/C22H26Cl2N2O4S/c1-16-6-2-3-10-21(16)3...
2424184,fffff0b6eea6,InChI=1S/C17H26N2O6S/c1-23-16-12-14(4-5-15(16)...


In [13]:
tmp = train.sample(1616107) # randomly choosing InChI strings from train

In [14]:
tmp

,image_id,InChI
704553,4a55b75d3503,InChI=1S/C18H18N2O2/c1-13-6-8-15(9-7-13)10-11-...
912558,604ae8809c63,"InChI=1S/C11H14N2O4S/c1-8(7-12)18(14,15)13-9-4..."
519992,36df81dcc05b,"InChI=1S/C22H35FO4/c1-4-5-14-22(2,3)20(25)13-1..."
779305,523c659bfd79,InChI=1S/C11H13Cl2NO/c1-11(4-5-14-7-11)15-8-2-...
1675050,b0d89905462b,InChI=1S/C18H21FN4S/c1-13(8-16-4-3-7-24-16)11-...
...,...,...
2334883,f68e106af93a,InChI=1S/C21H25NO3/c23-17-5-8-20-19(9-13-25-21...
166885,119509503bc3,"InChI=1S/C21H27ClN6O4/c1-20(2,3)15-14(22)17-25..."
1714230,b4fc12d8f987,InChI=1S/C26H29F5N8O3/c27-19(4-8-38-9-5-20(23(...
2354898,f8a6e8e90ef0,InChI=1S/C18H17N3O3/c1-3-15(18(23)24)21-16-5-4...


In [16]:
sample['InChI'][:] = tmp['InChI'][:]
sample

,image_id,InChI
0,00000d2a601c,InChI=1S/C18H18N2O2/c1-13-6-8-15(9-7-13)10-11-...
1,00001f7fc849,"InChI=1S/C11H14N2O4S/c1-8(7-12)18(14,15)13-9-4..."
2,000037687605,"InChI=1S/C22H35FO4/c1-4-5-14-22(2,3)20(25)13-1..."
3,00004b6d55b6,InChI=1S/C11H13Cl2NO/c1-11(4-5-14-7-11)15-8-2-...
4,00004df0fe53,InChI=1S/C18H21FN4S/c1-13(8-16-4-3-7-24-16)11-...
...,...,...
1616102,ffffcdb2e39e,InChI=1S/C21H25NO3/c23-17-5-8-20-19(9-13-25-21...
1616103,ffffcfddd770,"InChI=1S/C21H27ClN6O4/c1-20(2,3)15-14(22)17-25..."
1616104,ffffe4ab06b2,InChI=1S/C26H29F5N8O3/c27-19(4-8-38-9-5-20(23(...
1616105,ffffec4033ec,InChI=1S/C18H17N3O3/c1-3-15(18(23)24)21-16-5-4...


In [18]:
sample.to_csv('submissions/random_from_train.csv')

In [28]:
sample = pd.read_csv('submissions/random_from_train.csv')
sample

,Unnamed: 0,Unnamed: 0.1,image_id,InChI
0,0,0,00000d2a601c,InChI=1S/C18H18N2O2/c1-13-6-8-15(9-7-13)10-11-...
1,1,1,00001f7fc849,"InChI=1S/C11H14N2O4S/c1-8(7-12)18(14,15)13-9-4..."
2,2,2,000037687605,"InChI=1S/C22H35FO4/c1-4-5-14-22(2,3)20(25)13-1..."
3,3,3,00004b6d55b6,InChI=1S/C11H13Cl2NO/c1-11(4-5-14-7-11)15-8-2-...
4,4,4,00004df0fe53,InChI=1S/C18H21FN4S/c1-13(8-16-4-3-7-24-16)11-...
...,...,...,...,...
1616102,1616102,1616102,ffffcdb2e39e,InChI=1S/C21H25NO3/c23-17-5-8-20-19(9-13-25-21...
1616103,1616103,1616103,ffffcfddd770,"InChI=1S/C21H27ClN6O4/c1-20(2,3)15-14(22)17-25..."
1616104,1616104,1616104,ffffe4ab06b2,InChI=1S/C26H29F5N8O3/c27-19(4-8-38-9-5-20(23(...
1616105,1616105,1616105,ffffec4033ec,InChI=1S/C18H17N3O3/c1-3-15(18(23)24)21-16-5-4...


In [29]:
sample.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'image_id', 'InChI'], dtype='object')

In [31]:
sample.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
sample

,image_id,InChI
0,00000d2a601c,InChI=1S/C18H18N2O2/c1-13-6-8-15(9-7-13)10-11-...
1,00001f7fc849,"InChI=1S/C11H14N2O4S/c1-8(7-12)18(14,15)13-9-4..."
2,000037687605,"InChI=1S/C22H35FO4/c1-4-5-14-22(2,3)20(25)13-1..."
3,00004b6d55b6,InChI=1S/C11H13Cl2NO/c1-11(4-5-14-7-11)15-8-2-...
4,00004df0fe53,InChI=1S/C18H21FN4S/c1-13(8-16-4-3-7-24-16)11-...
...,...,...
1616102,ffffcdb2e39e,InChI=1S/C21H25NO3/c23-17-5-8-20-19(9-13-25-21...
1616103,ffffcfddd770,"InChI=1S/C21H27ClN6O4/c1-20(2,3)15-14(22)17-25..."
1616104,ffffe4ab06b2,InChI=1S/C26H29F5N8O3/c27-19(4-8-38-9-5-20(23(...
1616105,ffffec4033ec,InChI=1S/C18H17N3O3/c1-3-15(18(23)24)21-16-5-4...


In [32]:
sample.to_csv('submissions/random_from_train.csv', index=False)

In [33]:
sample = pd.read_csv('submissions/random_from_train.csv')
sample

,image_id,InChI
0,00000d2a601c,InChI=1S/C18H18N2O2/c1-13-6-8-15(9-7-13)10-11-...
1,00001f7fc849,"InChI=1S/C11H14N2O4S/c1-8(7-12)18(14,15)13-9-4..."
2,000037687605,"InChI=1S/C22H35FO4/c1-4-5-14-22(2,3)20(25)13-1..."
3,00004b6d55b6,InChI=1S/C11H13Cl2NO/c1-11(4-5-14-7-11)15-8-2-...
4,00004df0fe53,InChI=1S/C18H21FN4S/c1-13(8-16-4-3-7-24-16)11-...
...,...,...
1616102,ffffcdb2e39e,InChI=1S/C21H25NO3/c23-17-5-8-20-19(9-13-25-21...
1616103,ffffcfddd770,"InChI=1S/C21H27ClN6O4/c1-20(2,3)15-14(22)17-25..."
1616104,ffffe4ab06b2,InChI=1S/C26H29F5N8O3/c27-19(4-8-38-9-5-20(23(...
1616105,ffffec4033ec,InChI=1S/C18H17N3O3/c1-3-15(18(23)24)21-16-5-4...


In [34]:
!gzip -k submissions/random_from_train.csv

In [35]:
!ls -lh submissions/

total 298M
-rw-rw-r-- 1 alex alex 219M июн  8 10:46 random_from_train.csv
-rw-rw-r-- 1 alex alex  79M июн  8 10:46 random_from_train.csv.gz


With compressed .gz file scoring the Kaggle submission takes a lot longer.

Randomly choosing InChI string for test dataset from train dataset results in score 86.

In [37]:
from Levenshtein import distance as edit_dist

In [38]:
edit_dist(train['InChI'][0], train['InChI'][1])

99

In [39]:
train['InChI'][0]

'InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12(13)11(4)14/h5-7,9,11,14H,8H2,1-4H3'

In [40]:
train['InChI'][1]

'InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(10-14)11-17(23)19-15-4-5-18(24)21(15,3)9-7-16(19)20/h13-16,19H,4-11H2,1-3H3/t13-,14+,15+,16-,19-,20+,21+/m1/s1'

In [41]:
del train
del sample

## Choosing similar molecules by Carbon counts

Expect to already have predictions by our model about Carbon counts stored as a numpy file of integers.

In [4]:
submission = pd.read_csv('original_data/sample_submission.csv')      # all ['InChI'] will be changed below
c_counts_test  = np.load('data/results_c_test_from_kaggle.npy')
c_counts_train = np.load('data/pretrain2/atom_counts.npy')[:10**6,1] # choosing C counts
inchi_train = pd.read_csv('original_data/train_labels.csv')[:10**6]  # have precalculated C counts

In [5]:
submission

,image_id,InChI
0,00000d2a601c,InChI=1S/H2O/h1H2
1,00001f7fc849,InChI=1S/H2O/h1H2
2,000037687605,InChI=1S/H2O/h1H2
3,00004b6d55b6,InChI=1S/H2O/h1H2
4,00004df0fe53,InChI=1S/H2O/h1H2
...,...,...
1616102,ffffcdb2e39e,InChI=1S/H2O/h1H2
1616103,ffffcfddd770,InChI=1S/H2O/h1H2
1616104,ffffe4ab06b2,InChI=1S/H2O/h1H2
1616105,ffffec4033ec,InChI=1S/H2O/h1H2


In [6]:
inchi_train

,image_id,InChI
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...
3,000026b49b7e,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...
4,000026fc6c36,InChI=1S/C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7...
...,...,...
999995,6995b5216597,InChI=1S/C26H27FN4O/c27-21-8-3-7-20(15-21)24-1...
999996,6995d1050315,InChI=1S/C19H24N2O5/c1-11(2)14(15(22)20-10-9-1...
999997,6995d8bf18ad,InChI=1S/C15H18ClNO2S2/c1-9(2)12-8-19-15(21)17...
999998,6995db66d69b,InChI=1S/C20H19ClN6/c21-17-7-3-1-5-15(17)12-24...


In [7]:
submission.iat[0, 1] # expecting to see InChI string from the first row

'InChI=1S/H2O/h1H2'

In [8]:
inchi_train.iat[1, 1]

'InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(10-14)11-17(23)19-15-4-5-18(24)21(15,3)9-7-16(19)20/h13-16,19H,4-11H2,1-3H3/t13-,14+,15+,16-,19-,20+,21+/m1/s1'

In [11]:
from functools import lru_cache

In [31]:
@njit
def first_index(array, value):
    for index, val in np.ndenumerate(array):
        if val == value:
            return index[0]
    else:
        return -1

In [32]:
@lru_cache(maxsize=100)
def find_index_of_closest(c_count):
    """
    Function looks for the first precise match.
    If it's not found, looks for match for count +-1, count +-2.
    If match is not found returns 0 (dummy index).
    """
    res = first_index(c_counts_train, c_count)
    if res < 0:
        res = first_index(c_counts_train, c_count+1)
    if res < 0 and c_count > 0:
        res = first_index(c_counts_train, c_count-1)
    if res < 0:
        res = first_index(c_counts_train, c_count+2)
    if res < 0 and c_count > 1:
        res = first_index(c_counts_train, c_count-2)
    if res < 0:
        res = first_index(c_counts_train, c_count+3)
    if res < 0 and c_count > 2:
        res = first_index(c_counts_train, c_count-3)
    if res < 0:
        res = 0
    return res

In [33]:
for index in tqdm(range(submission.shape[0])):
    submission.iat[index, 1] = inchi_train.iat[find_index_of_closest(c_counts_test[index]), 1]

100%|██████████| 1616107/1616107 [01:02<00:00, 26034.04it/s]


In [34]:
submission

,image_id,InChI
0,00000d2a601c,InChI=1S/C7H6BrF3N2O2/c1-14-6-4(12)5(3(8)2-13-...
1,00001f7fc849,InChI=1S/C11H14N2O2/c1-7-12-6-9(5-10(14)15)11(...
2,000037687605,InChI=1S/C11H14N2O2/c1-7-12-6-9(5-10(14)15)11(...
3,00004b6d55b6,InChI=1S/C12H12O2S/c1-3-8-5-4-6-9-7-10(12(13)1...
4,00004df0fe53,InChI=1S/C8H10F3NS/c1-3-2-4(13)12-7-5(3)6(7)8(...
...,...,...
1616102,ffffcdb2e39e,InChI=1S/C18H19Cl2NO2/c1-12(2)9-10-23-14-6-3-5...
1616103,ffffcfddd770,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...
1616104,ffffe4ab06b2,InChI=1S/C14H20N2S/c1-2-13-7-4-8-16(13)10-11-5...
1616105,ffffec4033ec,InChI=1S/C11H14N2O2/c1-7-12-6-9(5-10(14)15)11(...


In [35]:
submission.to_csv('submissions/close_to_C_counts.csv', index=False)

In [36]:
!gzip -k submissions/close_to_C_counts.csv

Submitting InChI strings from train dataset similar to the test pictures in terms of the number of Carbon atoms gives score 74.

In [37]:
del c_counts_test
del c_counts_train

## Choosing similar molecules by 6 most frequent atoms count.

In [76]:
LENGTH = 10 ** 6 // 100
submission = pd.read_csv('original_data/sample_submission.csv')      # all ['InChI'] will be changed below
atoms_counts_test  = np.load('data/results_multiple_counts_from_kaggle.npy')

atoms_counts_train = np.zeros((LENGTH, 6),dtype=np.int32) # size=(LENGTH, 6 most frequent atoms)
atoms_counts_train[:,:4] = np.load('data/pretrain2/atom_counts.npy')[:LENGTH,1:(1+4)]
atoms_counts_train[:,4] = np.load('data/pretrain2/atom_counts.npy')[:LENGTH,7]
atoms_counts_train[:,5] = np.load('data/pretrain2/atom_counts.npy')[:LENGTH,8]

inchi_train = pd.read_csv('original_data/train_labels.csv')[:LENGTH]  # have precalculated C counts

In [77]:
atoms_counts_train

array([[13,  0,  1,  0,  1,  0],
       [21,  0,  4,  0,  0,  0],
       [24,  5,  4,  0,  0,  0],
       ...,
       [12,  3,  1,  0,  0,  0],
       [21,  3,  5,  0,  2,  0],
       [17,  0,  1,  0,  0,  1]], dtype=int32)

In [78]:
@njit
def distance(known, predicted):
    return np.sum(np.abs(known - predicted))

In [79]:
@njit
def find_index_of_closest1(atom_counts):
    dist  = 1e5
    index = 0
    for i in range(atoms_counts_train.shape[0]):
        d = distance(atoms_counts_train[i,:], atom_counts)
        if d < dist:
            dist = d
            index = i
    return index

In [81]:
%%time
for index in tqdm(range(submission.shape[0])):
    a = find_index_of_closest1(atoms_counts_test[index,:])
    submission.iat[index, 1] = inchi_train.iat[a, 1]

  1%|          | 15967/1616107 [00:13<22:47, 1170.35it/s]


KeyboardInterrupt: 

In [69]:
# example of joblib.Parallel
l = Parallel(n_jobs=6)(delayed(np.sqrt)(i ** 2) for i in range(10))
l

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [82]:
%%time
indices = Parallel(n_jobs=6)(delayed(find_index_of_closest1)(
                                    atoms_counts_test[index,:]) for index in range(submission.shape[0]))

CPU times: user 1min 55s, sys: 1.9 s, total: 1min 57s
Wall time: 4min 47s


In [83]:
for index in range(submission.shape[0]):
    submission.iat[index, 1] = inchi_train.iat[indices[index], 1]

In [84]:
submission

,image_id,InChI
0,00000d2a601c,InChI=1S/C10H14ClN3/c1-10(5-2-6-10)14-8(7-3-4-...
1,00001f7fc849,InChI=1S/C11H14ClN/c12-10-7-2-1-6-9(10)11(13)8...
2,000037687605,InChI=1S/C14H12ClFO/c1-9-6-7-10(12(15)8-9)14(1...
3,00004b6d55b6,InChI=1S/C15H14ClFN2O/c1-9-2-4-10(5-3-9)14(18)...
4,00004df0fe53,InChI=1S/C10H9ClO2/c11-9(10-8(12)6-13-10)7-4-2...
...,...,...
1616102,ffffcdb2e39e,InChI=1S/C19H28O/c1-4-6-7-8-9-13-16-20-19(17(3...
1616103,ffffcfddd770,"InChI=1S/C18H19ClO2/c1-18(2,3)17(20)14-6-4-13(..."
1616104,ffffe4ab06b2,"InChI=1S/C15H21ClO2/c1-11-5-7-15(18,8-6-11)14(..."
1616105,ffffec4033ec,InChI=1S/C12H13ClO2/c1-3-15-12(14)11(2)9-7-5-4...


In [85]:
submission.to_csv('submissions/close_to_atoms_counts.csv', index=False)

In [86]:
!gzip -k submissions/close_to_atoms_counts.csv

Submission result: score 73.